# LIbrerias ocupadadas

In [15]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
import string
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Funciones implementadas

In [22]:

def read_data(filename, ext):
    if ext == 'tsv':
        df = pd.read_table(filename)
        return df

def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
        u"\U0001F680-\U0001F6FF"  # Transporte y símbolos de mapa
        u"\U0001F1E0-\U0001F1FF"  # Banderas (iOS)
        u"\U00002700-\U000027BF"  # Otros símbolos diversos
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_non_alphabetic(tokens):
    return [word for word in tokens if word.isalpha()]

def process_text(text, lemmatizer, stemmer, stop_words):
    text = text.lower()
    text = remove_emojis(text)
    text = remove_punctuation(text)
    tokens = word_tokenize(text)
    tokens = remove_non_alphabetic(tokens)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [stemmer.stem(word) for word in tokens]
    processed_text = ' '.join(tokens)
    return processed_text

def process_data(df):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('spanish'))
    df['processed_text'] = df['tweet'].apply(lambda x: process_text(x, lemmatizer, stemmer, stop_words))
    return df

def get_MTD(df):
  vectorizer = CountVectorizer()
  X = vectorizer.fit_transform(df['processed_text'])
  tdm_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
  print(tdm_df)
  return tdm_df

# Reading Dataset

## Dataset
El dataset ocupado es de parte de TASS, una competencia que se realiza con los a;os con diferentes task, en particular aqui se ocupara el dataset de la task2 el cual consiste de tres archivos, se hara uso del archivo train.tsv en el cual en la siguiete linea se mostraran sus estadisticas.

In [11]:
df = read_data("train.tsv", "tsv")
print(df.head())

   id                                              tweet    label 
0   1  El Atlético resignado a perder HASHTAG 😔  http...  sadness 
1   2  Leer proporciona una mejor visión del mundo 🤓 ...      joy 
2   3  Amo a Arya Stark por encima de todas las cosas...      joy 
3   4  Gracias HASHTAG es increíble que una niña logr...   others 
4   5  Solo siento que hayamos perdido 24 escaños de ...  sadness 


In [12]:
df.count()

,0
id,5886
tweet,5886
label,5886


Se cuenta con 5886 ejemplos de tweets, cada uno con su etiquetada asociada

# Procesando el dataset

In [13]:
df = process_data(df)
print(df.columns)

Index(['id', 'tweet', 'label ', 'processed_text'], dtype='object')


Despues de procesado el dataset, se agrega una nueva columna con el texto procesado

In [14]:
cols = ['label ','processed_text']
print(df[cols])

         label                                      processed_text
0      sadness   atlético resignado perder hashtag httpstcoaiaa...
1          joy   leer proporciona mejor visión mundo hashtag ha...
2          joy   amo arya stark encima toda cosa gameofthron ha...
3       others   gracia hashtag increíbl niña logr liderar tan ...
4      sadness          solo siento perdido escaño cordura hashtag
...         ...                                                ...
5881    others   pequeño gesto pued hacer medio ambient hashtag...
5882  surprise   do coronacion celebraron hashtag napoleón únic...
5883     anger   mientra reflexionamo duro medio nueva guerra f...
5884   sadness   fachada catedr notr dame salvada podrá restaur...
5885       joy            sufriendo barsa si messi hashtag hashtag

[5886 rows x 2 columns]


## Obteniendo la matriz termino documento

In [32]:
 matrix_term_document = get_MTD(df)

      aaaaaaaaaah  aaaaaahsgajdkahakshah  aaahhhh  abajo  abandona  \
0               0                      0        0      0         0   
1               0                      0        0      0         0   
2               0                      0        0      0         0   
3               0                      0        0      0         0   
4               0                      0        0      0         0   
...           ...                    ...      ...    ...       ...   
5881            0                      0        0      0         0   
5882            0                      0        0      0         0   
5883            0                      0        0      0         0   
5884            0                      0        0      0         0   
5885            0                      0        0      0         0   

      abandonada  abandonado  abandonar  abandono  abanico  ...  órgano  \
0              0           0          0         0        0  ...       0   
1        

## Guardando la matriz en un csv

In [33]:
matrix_term_document.to_csv('matriz_termino_documento.csv', index=False)

### Cantidad de tokens para el primer documento

In [34]:
print(matrix_term_document.iloc[0].sum())

5


ANTES DEL PROCESO

In [35]:
print(df.iloc[0].tweet)

El Atlético resignado a perder HASHTAG 😔  https://t.co/AiaaGdxmQO


DESPUES DEL PROCESO

In [36]:
print(df.iloc[0].processed_text)

atlético resignado perder hashtag httpstcoaiaagdxmqo
